<a href="https://colab.research.google.com/github/5harad/DPI-617/blob/main/labs/disparate-treatment-exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Law, Order, and Algorithms**
## Statistical tests for disparate treatment

**Getting started**

Before you start, create a copy of this Jupyter notebook in your own Google Drive by clicking `Copy to Drive` in the menubar. If you do not do this your work will not be saved! 

Remember to save your work frequently by pressing command-S or clicking File > Save in the menubar. 

We recommend completing this problem set in Google Chrome.

Run the cell below to load the `tidyverse` library and set some formatting options.

In [ ]:
# Some initial setup

# load libraries
library(tidyverse)

# Set some formatting options
options(digits = 3, repr.matrix.max.rows = 10, repr.matrix.max.cols = 100)
theme_set(theme_bw())


##Background

In this lab, we'll explore competing understandings of *disparate treatment* in frisk decisions, using the NYC stop-and-frisk data. 

We'll explore limitations of thinking about disparate treatment as the "causal effect of race" on decisions, and consider an alternative view that focuses considerations of intent and animus.

## The data

This week, we will use the same data that we used a couple of weeks ago: a sample of stops in NYC, recorded on a 
[UF-250 form][uf250_link].

Run the cell below to load the data. For simplicity, we'll work with a subset of the columns, and we'll limit to stops where there is no missing information (i.e., every variable has a value).

[uf250_link]: https://www.prisonlegalnews.org/media/publications/Blank%20UF-250%20Form%20-%20Stop%2C%20Question%20and%20Frisk%20Report%20Worksheet%2C%20NYPD%2C%202016.pdf

In [ ]:
# Load data
fname <- "https://github.com/5harad/DPI-617/blob/main/data/sqf_sample.rds?raw=true"
stop_df <- readRDS(url(fname))

# Select the columns we'll use in the lab today, and exclude missing observations
stop_df <- stop_df %>%
  select('found_weapon', 'frisked', 'suspected_crime', 'precinct',
         'month', 'day', 'year',
         'inside_outside', 'location_housing',
         'radio_run', 'officer_uniform', 'observation_period',
         starts_with('subject_'), -subject_hispanic, -subject_dob,
         starts_with('stopped_bc_'),
         starts_with('additional_')) %>%
  filter(subject_race != 'other') %>%
  mutate(subject_race = fct_relevel(subject_race, 'white')) %>%
  na.omit()

stop_df

We start by comparing frisk rates of racial minorities to frisk rates of white individuals. In other words, are white people who are stopped frisked more or less often than racial minorities who are stopped? 

### Exercise 1: Computing raw disparities

Compute the proportion of stopped individuals who are frisked, by race group. How do you interpret the results? Do you take your results as stong evidence of disparate treatment?


In [ ]:
# WRITE CODE HERE


Even if one race group has higher frisk rates than another, one could make the argument that officers might have a legitimate reason to frisk certain types of individuals more often and that reason happens to correlate with race. Such a reason is often called a *confounding variable*. In the presence of a confounding variable our raw differences in frisk rates may be a poor measure of disparate treatment.

### Exercise 2: Dealing with confounders

One of the most commonly suspected crimes is *cpw*, criminal possession of a weapon. Given that the primary justification of a frisk is concern for officer safety, one could argue that it is reasonable for an officer to frisk individuals whom they have stopped under suspicion of criminal possession of weapons. It would thus make sense for cpw stops to have higher frisk rates. And if a larger proportion of stops of minorities were cpw stops, then we might correspondingly expect frisk rates to be higher among minority groups, even in the absence of disparate treatment, due to this confounder.

To test this theory, first create a new binary (`TRUE`/`FALSE`) column in `stop_df` named `is_cpw` that is `TRUE` if `suspected_crime` is `cpw`. 

In [ ]:
# WRITE CODE TO ADD is_cpw column HERE


Now answer the following:
* What proportion of stops were cpw stops, by `subject_race`?
* Among cpw stops, what are the frisk rates by `subject_race`? Among non-cpw stops, what are the frisk rates by `subject_race`?
(*Hint:* For the two tasks above, your new `is_cpw` column will come in handy!)
* Does `is_cpw` seem like a confounding variable? Do there still appear to be racial disparities in frisk rates even after accounting for whether a stop was for cpw or not?

In [ ]:
# WRITE CODE HERE


In the above analysis, we computed the frisk rates by race after *adjusting for cpw*. Another way to do this type of adjustment is with a statistical model. 

The two snippets of code below correspond to two statistical model: the first computes frisk rates by race; and second computes frisk rates by race white adjusting for whether the basis of a stop was criminal possession of a weapon (cpw).

In [ ]:
baseline_model <- lm(frisked ~ subject_race, data = stop_df)
summary(baseline_model)

In [ ]:
model_with_cpw <- lm(frisked ~ subject_race + is_cpw, data = stop_df)
summary(model_with_cpw)

One of the advantages of using statistical models like those above, is that they let us compute disparities after adjusting for a a variety of factors.

### Exercise 3: Estimating disparate treatment


Explore different models to estimate disparate treatment. 

Remember that we discussed two different understandings of disparate treatment. The first aims to estimate the "causal effect of race" on decisions, a popular objective, but one that is on statistically shaky ground. The second understanding of disparate treament centers on the idea of intent or animus. 

How do these two different understandings of disparate treatment effect the statistical model you use?

In [ ]:
# in a "causal" model, adjust for all factors that are not 
# clear proxies for race

# WRITE CODE HERE


In [ ]:
# in an intent-based model, we need to be more judicious about what we adjust for,
# though there is often ambiguity in exactly what we do.

# WRITE CODE HERE
